In [389]:
# imports
import re
import string
import time
import requests
import regex as re
from bs4 import BeautifulSoup
import csv
import os
from os import path
import pandas as pd
from datetime import datetime, timezone

In [390]:
# Criar arquivo CSV
def baseArchive():
    basePath = os.path.join(os.getcwd(), 'db', 'dataProjs.csv')
    os.makedirs(os.path.dirname(basePath), exist_ok=True)
    if not os.path.exists(basePath):
        with open(basePath, 'w', encoding='utf-8') as baseArchive:
            baseArchive.write('titulo,resumo,URL\n')  # Cabeçalho compatível com os dados
    print(f"Arquivo base criado ou já existente em: {basePath}")
    return basePath


In [391]:
# limpeza
def cleanSentence(phrase):
    # Convertendo para minúsculas
    phrase = phrase.lower()

    # Removendo URLs
    phrase = re.sub(r'http\S+|www\S+|https\S+', '', phrase, flags=re.MULTILINE)

    # Removendo números
    phrase = re.sub(r'\d+', '', phrase)

    # Removendo pontuação
    phrase = phrase.translate(str.maketrans('', '', string.punctuation))

    # Removendo espaços extras
    phrase = phrase.strip()

    # Removendo múltiplos espaços
    phrase = re.sub(r'\s+', ' ', phrase)

    return phrase
    

In [392]:
def get_data(url_model):
    print(f"Obtendo artigo em: {url_model}")
    response = requests.get(url_model)
    if response.status_code != 200:
        print(f"Erro ao acessar curso em {url_model}: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    data_base_news = []

    content_seccion = soup.find('div', class_='bastian-page')

    articles_seccion = content_seccion.find_all('div', class_='bastian-feed-item')
    print(articles_seccion)

    for seccion in articles_seccion: 
        title_seccion = seccion.find('div', class_='_evt')
        title = title_seccion.find('p')

        URL = seccion.find('a')['href']

        resume_seccion = seccion.find('div', class_='feed-post-body-resumo')
        resume = resume_seccion.find('p')
        
        title_text = title.text.strip()
        resume_text = resume.text.strip()
        resume_clean = cleanSentence(resume_text)
            
        data_base_news.append({
            'titulo': title_text,
            'resumo': resume_clean,
            'URL': URL
        })
        print(f'coletado! \nTITULO: {title_text}\nRESUMO: {resume_text}\nURL: {URL}')
        
        
    return data_base_news



In [393]:
def saveData(data, filename=None):
    if filename is None:
        filename = baseArchive()

    df = pd.DataFrame(data)

    try:
        # Caso o arquivo já exista
        if os.path.exists(filename):
            existing_df = pd.read_csv(filename)
            
            # Combina os novos dados com os existentes, sem remover duplicatas
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df.to_csv(filename, index=True, index_label='index')
        else:
            # Caso seja o primeiro salvamento
            df.to_csv(filename, index=True, index_label='index')

        print(f"Dados salvos no arquivo '{filename}' com sucesso!")

    except Exception as e:
        print(f"Erro ao salvar dados: {e}")

In [394]:
if __name__ == "__main__":
    url_base = "https://g1.globo.com/fato-ou-fake/"
    data = get_data(url_base)

    if data:
        saveData(data)
    else:
        print(f'nenhum dado encontrado')

Obtendo artigo em: https://g1.globo.com/fato-ou-fake/
[<div class="bastian-feed-item" data-index="1"><div class="feed-post bstn-item-shape type-materia" id="8c023bef966c28c69f24805c689b1656"><div class="feed-post-body"><div class="feed-post-header"></div><div class="feed-post-body-title gui-color-primary gui-color-hover"><div class="_evt"><h2><a class="feed-post-link gui-color-primary gui-color-hover" href="https://g1.globo.com/fato-ou-fake/noticia/2025/02/14/e-fake-foto-que-mostra-alexandre-de-moraes-usando-bone-da-usaid.ghtml"><p elementtiming="text-ssr">É #FAKE foto que mostra Alexandre de Moraes usando boné da USAID</p></a></h2></div></div><div class="feed-post-body-resumo"><p elementtiming="text-ssr">Imagem criada com inteligência artificial mostra ministro do STF usando acessório com logotipo da Agência dos Estados Unidos para o Desenvolvimento Internacional. Ela passou a circular dias depois de Trump anunciar que encerraria as atividades do órgão.</p></div><div class="feed-media